<a href="https://colab.research.google.com/github/VineetKiragi/CMPE-257/blob/main/CMPE_257_Question_Answering_Task_Rahul_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Make sure you have a GPU running
!nvidia-smi

Wed Nov 30 20:07:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 34.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-dndz8u2j/farm-haystack_411d07c093d2409b973d2e7f3a693ed3
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-dndz8u2j/farm-haystack_411d07c093d2409b973d2e7f3a693ed3
  Resolved https://github.com/deepset-ai/haystack.git to commit adb580b6b7b1960fa28df85d8544722b0c1e135a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.

In [5]:
from haystack.nodes import FARMReader


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "data/"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"


In [7]:
reader.train(data_dir=data_dir, train_filename="/content/drive/MyDrive/train.json", use_gpu=True, n_epochs=10, save_dir="my_model",)

Train epoch 9/9 (Cur. train loss: 0.5407): 100%|██████████| 44/44 [00:09<00:00,  4.56it/s]


In [8]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
reader.save(directory="my_model")

In [9]:
# If you want to load it at a later point, just do:
new_reader = FARMReader(model_name_or_path="my_model")

In [10]:
reader_eval_results = new_reader.eval_on_file("data/", "/content/drive/MyDrive/test.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 2/2 [00:00<00:00,  2.76it/s]


In [11]:
reader_eval_results

{'EM': 22.47191011235955,
 'f1': 54.187797553506456,
 'top_n_accuracy': 80.89887640449437,
 'top_n': 4,
 'EM_text_answer': 15.0,
 'f1_text_answer': 50.28392477827593,
 'top_n_accuracy_text_answer': 78.75,
 'top_n_EM_text_answer': 15.0,
 'top_n_f1_text_answer': 51.79457926838248,
 'Total_text_answer': 80,
 'EM_no_answer': 88.88888888888889,
 'f1_no_answer': 88.88888888888889,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 9}

In [12]:
context = '''Visakhapatnam is a port city and industrial center in the Indian state of Andhra Pradesh, on the Bay of Bengal. It's known for its many beaches, including Ramakrishna Beach, home to a preserved submarine at the Kursura Submarine Museum. Nearby are the elaborate Kali Temple and the Visakha Museum, an old Dutch bungalow housing local maritime and historical exhibits.'''

In [13]:
new_reader.predict_on_texts("How is the weather of the city?",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 51.09 Batches/s]


{'query': 'How is the weather of the city?',
 'no_ans_gap': 5.532699108123779,
 'answers': [<Answer {'answer': "It's known for its many beaches, including Ramakrishna Beach, home to a preserved submarine at the Kursura Submarine Museum.", 'type': 'extractive', 'score': 0.31196480989456177, 'context': "y of Bengal. It's known for its many beaches, including Ramakrishna Beach, home to a preserved submarine at the Kursura Submarine Museum. Nearby are t", 'offsets_in_document': [{'start': 112, 'end': 236}], 'offsets_in_context': [{'start': 13, 'end': 137}], 'document_id': '180afd6b388ea3555a775d177f6e691b', 'meta': {}}>]}

###Inference Using Pipeline


In [14]:
from haystack import Pipeline, Document
from haystack.utils import print_answers
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])
res = p.run(
    query="What is the historic importance of the city?", documents=[Document(content=context)]
)
print_answers(res,details="medium")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 55.33 Batches/s]


Query: What is the historic importance of the city?
Answers:
[   {   'answer': "It's known for its many beaches, including Ramakrishna "
                  'Beach, home to a preserved submarine at the Kursura '
                  'Submarine Museum',
        'context': "ay of Bengal. It's known for its many beaches, including "
                   'Ramakrishna Beach, home to a preserved submarine at the '
                   'Kursura Submarine Museum. Nearby are ',
        'score': 0.5704846978187561}]


In [15]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split

In [16]:
null = "null"
false=False
true = True

with open("/content/drive/MyDrive/merged_file.json", encoding='UTF-8') as f:
    a= json.load(f)

In [17]:
a

{'data': [{'paragraphs': [{'qas': [{'question': 'In which state is Los Angeles?',
       'id': 597553,
       'answers': [{'answer_id': 647225,
         'document_id': 1202512,
         'question_id': 597553,
         'text': 'California',
         'answer_start': 36,
         'answer_end': 46,
         'answer_category': None}],
       'is_impossible': False},
      {'question': 'What more can be told about the city?',
       'id': 597555,
       'answers': [{'answer_id': 647227,
         'document_id': 1202512,
         'question_id': 597555,
         'text': 'TCL Chinese Theatre displays celebrities’ hand- and footprints, the Walk of Fame honors thousands of luminaries and vendors sell maps to stars’ homes',
         'answer_start': 266,
         'answer_end': 415,
         'answer_category': None}],
       'is_impossible': False},
      {'question': 'What are some iconic thing about the city?',
       'id': 597559,
       'answers': [{'answer_id': 647232,
         'document_id': 12

In [18]:
p = []
q=[]
ans = []
astart = []
aend= []
for i in a["data"]:
    for j in i["paragraphs"][0]["qas"]:
        #print(j)
        if(len(j["answers"])!=0):
          astart.append(j["answers"][0]["answer_start"])

          p.append(i["paragraphs"][0]["context"])
          q.append(j["question"])
          
          ans.append(j["answers"][0]["text"])
          aend.append(j["answers"][0]["answer_end"])
        
df = pd.DataFrame()
df["context"] = p
df["questions"] = q
df["answer"] = ans
df["answer_start"] = astart
df["answer_end"] = aend


In [19]:
df


,context,questions,answer,answer_start,answer_end
0,Los Angeles is a sprawling Southern California city and the center of the na...,In which state is Los Angeles?,California,36,46
1,Los Angeles is a sprawling Southern California city and the center of the na...,What more can be told about the city?,"TCL Chinese Theatre displays celebrities’ hand- and footprints, the Walk of ...",266,415
2,Los Angeles is a sprawling Southern California city and the center of the na...,What are some iconic thing about the city?,"Paramount Pictures, Universal and Warner Brothers",161,210
3,New York City comprises 5 boroughs sitting where the Hudson River meets the ...,What are some iconic thing about the city?,Empire State Building and sprawling Central Park,269,318
4,New York City comprises 5 boroughs sitting where the Hudson River meets the ...,What could be said the geography of the city?,comprises 5 boroughs sitting where the Hudson River meets the Atlantic Ocean,14,90
...,...,...,...,...,...
495,"Oxnard is a seaside city west of Los Angeles, in California. Its central Car...",What are the popular landmarks in this city?,Its central Carnegie Art Museum displays contemporary California art in a fo...,61,211
496,"Oxnard is a seaside city west of Los Angeles, in California. Its central Car...",What is this city known for?,"On the Pacific Coast, Oxnard Beach Park has a wide, sandy shoreline and a gr...",213,412
497,Columbus is a city in western Georgia. The Chattahoochee Riverwalk is a long...,Where is the city located?,Columbus is a city in western Georgia,0,37
498,Columbus is a city in western Georgia. The Chattahoochee Riverwalk is a long...,What are the popular landmarks in this city?,The Chattahoochee Riverwalk is a long promenade for walking or biking along ...,39,138
